<a href="https://colab.research.google.com/github/greyhound101/gan_segmentation_FE/blob/main/seeing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

In [ ]:

import numpy as np
import torch
import os
from torch import nn
from torch import optim
from torch.nn import functional as F

class Discriminator(nn.Module):
    def __init__(self, channel=512):
        super(Discriminator, self).__init__()        
        self.channel = channel
        n_class = 1
        
        self.conv1 = nn.Conv3d(1, channel//8, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv3d(channel//8, channel//4, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm3d(channel//4)
        self.conv3 = nn.Conv3d(channel//4, channel//2, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm3d(channel//2)
        self.conv4 = nn.Conv3d(channel//2, channel, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm3d(channel)
        
        self.conv5 = nn.Conv3d(channel, n_class, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x, _return_activations=False):
        h1 = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        h2 = F.leaky_relu(self.bn2(self.conv2(h1)), negative_slope=0.2)
        h3 = F.leaky_relu(self.bn3(self.conv3(h2)), negative_slope=0.2)
        h4 = F.leaky_relu(self.bn4(self.conv4(h3)), negative_slope=0.2)
        h5 = self.conv5(h4)
        output = h5

        return output
    

class Generator(nn.Module):
    def __init__(self, noise:int=1000, channel:int=64):
        super(Generator, self).__init__()
        _c = channel
        
        self.noise = noise
        self.fc = nn.Linear(1000,512*4*4*4)
        self.fc.requires_grad_=False
        self.bn1 = nn.BatchNorm3d(_c*8)
        self.bn1.requires_grad_=False
        
        self.tp_conv2 = nn.Conv3d(_c*8, _c*4, kernel_size=3, stride=1, padding=1, bias=False)
        self.tp_conv2.requires_grad_=False
        self.bn2 = nn.BatchNorm3d(_c*4)
        self.bn2.requires_grad_=False
        
        self.tp_conv3 = nn.Conv3d(_c*4, _c*2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm3d(_c*2)
        
        self.tp_conv4 = nn.Conv3d(_c*2, _c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm3d(_c)
        
        self.tp_conv5 = nn.Conv3d(_c, 1, kernel_size=3, stride=1, padding=1, bias=False)
        
    def forward(self, noise):
        noise = noise.view(-1, 1000)
        h = self.fc(noise)
        h = h.view(-1,512,4,4,4)
        h = F.relu(self.bn1(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv2(h)
        h = F.relu(self.bn2(h))
        
        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv3(h)
        h = F.relu(self.bn3(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv4(h)
        h = F.relu(self.bn4(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv5(h)

        h = F.tanh(h)

        return h
g=torch.load('/content/gdrive/MyDrive/generator_199.pth',map_location='cpu')
G1 = Generator()

G1.load_state_dict(g['model_state_dict'])


In [ ]:

from skimage.transform import resize
img_deps=512
img_rows=512
img_cols=32

In [ ]:
import gc
from tqdm import tqdm
for i in tqdm(range(28)):
  noise = torch.randn((1, 1000, 1, 1, 1))
  fake_images = G1(noise)
  fake_images=fake_images[0,0].detach().numpy()
  img=np.round(fake_images[:,:,:32]*255)
  livertumor=np.round(fake_images[:,:,32:]*2)
  livertumor[livertumor<0]=0
  img[img < -200] = -200
  img[img > 250] = 250
  img = np.array(img, dtype='float32')
  cropp_tumor = resize(livertumor, (img_deps,img_rows,img_cols), order=0, mode='edge', cval=0, clip=True, preserve_range=True)
  cropp_img   = resize(img, (img_deps,img_rows,img_cols), order=3, mode='constant', cval=0, clip=True, preserve_range=True)
  np.save('/content/gdrive/MyDrive/test_'+str(i)+'.npy',[cropp_tumor,cropp_img])
  del([noise,fake_images,img,livertumor,cropp_tumor,cropp_img])
  gc.collect()